# NLP Data Preprocess Examples
- [Datasets](#datasets)
- [Basic](#basic)
- [Word Embedding](#word-embedding)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import nltk
import konlpy
import gensim
import tensorflow

import numpy as np
import pandas as pd
import sklearn
import re

## Datasets
- [20 Newsgroups Dataset](#20-newsgroups-dataset)
- [Naver Movie Review Dataset](#naver-movie-review-dataset)

### 20 Newsgroups Dataset

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

documents = [re.sub("[^a-zA-Z ]", ' ', document) for document in documents]
documents = [document.lower() for document in documents if len(document.split())>3]

stop_words = stopwords.words('english')
pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
tokenized = [pattern.sub('', document).split() for document in documents]
tokenized = [document for document in tokenized if len(document)>1]
print('Samples:', len(tokenized))

Samples: 10930


### Naver Movie Review Dataset

In [ ]:
from urllib import request
from konlpy.tag import Okt

request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt', filename='ratings.txt')
data = pd.read_table('ratings.txt')
data['document'] = data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
data = data[data['document'].notna()]

okt = Okt()
stop_words = {'의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'}
tokenized = list()

for sentence in data['document']:
    tokenized_sentence = okt.morphs(sentence, stem=True)
    tokenized_sentence = [word for word in tokenized_sentence if word not in stop_words]
    tokenized.append(tokenized_sentence)
print('Samples:', len(tokenized))

### Tokenized Dataset Encoding

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized)

word2idx = tokenizer.word_index
idx2word = {value:key for key,value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized)

## Basic
- [Tokenization](#tokenization)
- [Stemming](#stemming)
- [Lemmatization](#lemmatization)
- [Stopword](#stopword)
- [Integer Encoding](#integer-encoding)
- [Padding](#padding)
- [One-Hot Encoding](#one-hot-encoding)
- [Korean Tools](#korean-tools)

### Tokenization

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

text = "I am actively looking for Ph.D. students. and you are a Ph.D. student."
tokenized_sentence = word_tokenize(text)

print('Word tokenization:', tokenized_sentence)
print('POS tagging:', pos_tag(tokenized_sentence))

Word tokenization: ['I', 'am', 'actively', 'looking', 'for', 'Ph.D.', 'students', '.', 'and', 'you', 'are', 'a', 'Ph.D.', 'student', '.']
POS tagging: [('I', 'PRP'), ('am', 'VBP'), ('actively', 'RB'), ('looking', 'VBG'), ('for', 'IN'), ('Ph.D.', 'NNP'), ('students', 'NNS'), ('.', '.'), ('and', 'CC'), ('you', 'PRP'), ('are', 'VBP'), ('a', 'DT'), ('Ph.D.', 'NNP'), ('student', 'NN'), ('.', '.')]


In [ ]:
from nltk.tokenize import sent_tokenize

text = "His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to make sure no one was near."
print('Sentence Tokenization:', sent_tokenize(text))

Sentence Tokenization: ['His barber kept his word.', 'But keeping such a huge secret to himself was driving him crazy.', 'Finally, the barber went up a mountain and almost to the edge of a cliff.', 'He dug a hole in the midst of some reeds.', 'He looked about, to make sure no one was near.']


In [ ]:
from konlpy.tag import Okt

okt = Okt()
text = '열심히 코딩한 당신, 연휴에는 여행을 가봐요'

print('OKT 형태소 분석:', okt.morphs(text))
print('OKT 품사 태깅:', okt.pos(text))
print('OKT 명사 추출:', okt.nouns(text))

OKT 형태소 분석: ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
OKT 품사 태깅: [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
OKT 명사 추출: ['코딩', '당신', '연휴', '여행']


### Stemming

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize

porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
words = ['formalize', 'allowance', 'electricical']

print('Original:'.ljust(18,' '), words)
print('Porter Stemmer:'.ljust(18,' '), [porter_stemmer.stem(w) for w in words])
print('Lancaster Stemmer:'.ljust(18,' '), [lancaster_stemmer.stem(w) for w in words])

Original:          ['formalize', 'allowance', 'electricical']
Porter Stemmer:    ['formal', 'allow', 'electric']
Lancaster Stemmer: ['form', 'allow', 'elect']


### Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print('Original:'.ljust(14,' '), words)
print('Lemmatization:'.ljust(14,' '), [lemmatizer.lemmatize(word) for word in words])

Original:      ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']
Lemmatization: ['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


### Stopword

In [ ]:
from nltk.corpus import stopwords

stop_words_list = stopwords.words('english')
print('Stopwords:',stop_words_list[:10])

Stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [ ]:
example = "Family is not an important thing. It's everything."
stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(example)

print('Origianl:'.ljust(9,' '), word_tokens) 
print('Result:'.ljust(9,' '), [word for word in word_tokens if word not in stop_words])

Origianl: ['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything', '.']
Result:   ['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

raw_text = 'A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain.'
stop_words = set(stopwords.words('english'))

sentences = sent_tokenize(raw_text)
tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
preprocessed_sentences = list()

for sentence in tokenized_sentences:
    word_tokens = [word.lower() for word in sentence if len(word) > 2]
    word_tokens = [word for word in word_tokens if word not in stop_words]
    preprocessed_sentences.append(word_tokens)

### Integer Encoding

In [ ]:
from collections import Counter
from nltk import FreqDist

words = np.hstack(preprocessed_sentences)
vocab_size = 5
vocab = dict(Counter(words).most_common(vocab_size))
# vocab = dict(FreqDist(words).most_common(vocab_size))
print('Word counts:', vocab)

word_to_index = {word:i for i,word in enumerate(vocab.keys(),1)}
word_to_index['OOV'] = len(word_to_index)+1 # Out-Of-Vocabulary 토큰 추가
print('Word index:', word_to_index)

encoded_sentences = list()
for sentence in preprocessed_sentences:
    encoded_sentence = [word_to_index[word] if word in word_to_index else word_to_index['OOV'] for word in sentence]
    encoded_sentences.append(encoded_sentence)

print('Encoded sentences:', encoded_sentences)

Word counts: {'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3}
Word index: {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}
Encoded sentences: [[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size = 5
tokenizer = Tokenizer(num_words=vocab_size+2, oov_token='OOV')
tokenizer.fit_on_texts(preprocessed_sentences)
encoded = tokenizer.texts_to_sequences(preprocessed_sentences)

# num_words는 texts_to_sequences()에만 적용
print('Word counts:', dict(tokenizer.word_counts))
print('Word index:', tokenizer.word_index)
print('Encoded sentences:', encoded)

Word counts: {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}
Word index: {'OOV': 1, 'barber': 2, 'secret': 3, 'huge': 4, 'kept': 5, 'person': 6, 'word': 7, 'keeping': 8, 'good': 9, 'knew': 10, 'driving': 11, 'crazy': 12, 'went': 13, 'mountain': 14}
Encoded sentences: [[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


### Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded = pad_sequences(encoded, padding='post', truncating='post', maxlen=5)
padded

array([[2, 6, 0, 0, 0],
       [2, 1, 6, 0, 0],
       [2, 4, 6, 0, 0],
       [1, 3, 0, 0, 0],
       [3, 5, 4, 3, 0],
       [4, 3, 0, 0, 0],
       [2, 5, 1, 0, 0],
       [2, 5, 1, 0, 0],
       [2, 5, 3, 0, 0],
       [1, 1, 4, 3, 1],
       [2, 1, 4, 1, 0]], dtype=int32)

### One-Hot Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical

print('endoced:', encoded[0])

one_hot = to_categorical(encoded[0])
one_hot

endoced: [2, 6]


array([[0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

### Korean Tools

In [ ]:
sentence = str()

# PyKoSpacing
# 띄어쓰기 교정
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git

from pykospacing import Spacing

spacing = Spacing()
kospacing_sent = spacing(sentence)

# Py-Hanspell
# 네이버 한글 맞춤법 검사기
# pip install git+https://github.com/ssut/py-hanspell.git

from hanspell import spell_checker

spelled_sent = spell_checker.check(sentence)
hanspell_sent = spelled_sent.checked

# SOYNLP
# 학습 기반 토크나이저, 반복되는 문자 정제
# pip install soynlp

## Word Embedding
- [DTM](#dtm)
- [TF-IDF](#tf-idf)
- [Word2Vec](#word2vec)

### DTM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love', 'I like you', 'what should I do ']
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ['you know I want your love', 'I like you', 'what should I do ']
vector = TfidfVectorizer()

print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### Word2Vec

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=tokenized, window=5, min_count=5, workers=4, sg=0)

In [ ]:
model_result = model.wv.most_similar('like')
print(model_result)

[('nice', 0.6341156363487244), ('better', 0.5764698386192322), ('probably', 0.572699248790741), ('prefer', 0.5555704236030579), ('think', 0.5463138222694397), ('want', 0.5313563346862793), ('looking', 0.5295560359954834), ('suspiciously', 0.5255772471427917), ('bad', 0.5236798524856567), ('guess', 0.5195480585098267)]


In [ ]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v') # 모델 로드